In [6]:
import os
import json

import folium

# Chemin vers le dossier contenant les fichiers JSON
dossier = "/Users/mazziap/Developer/isochrone_map/isochrone_map/data/geojson_mp_data"

# Liste pour stocker tous les JSON
json_mp_data = []

# Parcours de tous les fichiers du dossier
for nom_fichier in os.listdir(dossier):
    chemin_complet = os.path.join(dossier, nom_fichier)
    timing = chemin_complet[:-8].split("_")[-1]
    with open(chemin_complet, "r", encoding="utf-8") as f:
        data = json.load(f)
        json_mp_data.append((data, timing))

# Affichage ou utilisation
print(f"Nombre de fichiers JSON chargés : {len(json_mp_data)}")

Nombre de fichiers JSON chargés : 10


In [7]:
# Chemin vers le dossier contenant les fichiers JSON
dossier = "/Users/mazziap/Developer/isochrone_map/isochrone_map/data/geojson_total"

# Liste pour stocker tous les JSON
json_total = []

# Parcours de tous les fichiers du dossier
for nom_fichier in os.listdir(dossier):
    chemin_complet = os.path.join(dossier, nom_fichier)
    timing = chemin_complet[:-8].split("_")[-1]
    with open(chemin_complet, "r", encoding="utf-8") as f:
        data = json.load(f)
        json_total.append((data, timing))

# Affichage ou utilisation
print(f"Nombre de fichiers JSON chargés : {len(json_total)}")

Nombre de fichiers JSON chargés : 10


In [12]:
from isochrone_map.constants import PATH_BIKING_GEO


m = folium.Map(location=[48.8566, 2.3522], zoom_start=12)

# Ajout des polygones isochrones
for geojson, time in json_total:
    folium.GeoJson(
        geojson,
        name=f"Metro Total {time} min",
        style_function=lambda x: {
            "fillColor": "#3186cc",
            "color": "#3186cc",
            "weight": 2,
            "fillOpacity": 0.3,
        },
    ).add_to(m)

# Ajout des polygones isochrones
for geojson, time in json_mp_data:
    folium.GeoJson(
        geojson,
        name=f"Metro MP DATA {time} min",
        style_function=lambda x: {
            "fillColor": "#78134c",
            "color": "#b531cc",
            "weight": 2,
            "fillOpacity": 0.3,
        },
    ).add_to(m)

with open(PATH_BIKING_GEO, "r", encoding="utf-8") as f:
    geojson = json.load(f)

# Ordonner du plus long au plus court pour que le court soit au-dessus
feats = sorted(
    geojson["features"], key=lambda f: f["properties"].get("value", 0), reverse=True
)

for idx, feat in enumerate(feats):
    minutes_val = int(round(feat["properties"].get("value", 0) / 60))
    folium.GeoJson(
        data=feat,
        name=f"Biking {minutes_val} min",
        style_function=lambda x: {
            "fillColor": "#137815",
            "color": "#40cc31",
            "weight": 2,
            "fillOpacity": 0.3,
        },
    ).add_to(m)

folium.LayerControl(collapsed=False).add_to(m)

m.save("isochrone_transport_paris.html")

In [13]:
import json
import folium
from isochrone_map.constants import PATH_BIKING_GEO


def add_isochrones(
    map_obj,
    data,
    name_prefix,
    color_fill,
    color_edge,
    order_by_value=True,
    value_key="value",
    reverse=True,
):
    """
    Ajoute des isochrones à une carte Folium.

    Args:
        map_obj: objet folium.Map
        data: liste de tuples (geojson, time) ou dict geojson avec "features"
        name_prefix: préfixe pour le nom de la couche
        color_fill: couleur de remplissage
        color_edge: couleur du contour
        order_by_value: si True, trie les features par 'value_key'
        value_key: clé dans properties pour le tri
        reverse: True si on veut du plus grand au plus petit
    """
    # Si data est un dict avec des features (comme pour biking)
    if isinstance(data, dict) and "features" in data:
        features = data["features"]
        if order_by_value:
            features = sorted(
                features,
                key=lambda f: f["properties"].get(value_key, 0),
                reverse=reverse,
            )
        for feat in features:
            minutes_val = int(round(feat["properties"].get(value_key, 0) / 60))
            folium.GeoJson(
                data=feat,
                name=f"{name_prefix} {minutes_val} min",
                style_function=lambda x: {
                    "fillColor": color_fill,
                    "color": color_edge,
                    "weight": 2,
                    "fillOpacity": 0.3,
                },
            ).add_to(map_obj)
    else:
        # On suppose que c'est une liste de tuples (geojson, time)
        for geojson, time in data:
            folium.GeoJson(
                geojson,
                name=f"{name_prefix} {time} min",
                style_function=lambda x: {
                    "fillColor": color_fill,
                    "color": color_edge,
                    "weight": 2,
                    "fillOpacity": 0.3,
                },
            ).add_to(map_obj)


# Création de la carte
m = folium.Map(location=[48.8566, 2.3522], zoom_start=12)

# Ajout des isochrones
add_isochrones(m, json_total, "Metro Total", "#3186cc", "#3186cc")
add_isochrones(m, json_mp_data, "Metro MP DATA", "#78134c", "#b531cc")

# Biking
with open(PATH_BIKING_GEO, "r", encoding="utf-8") as f:
    biking_geojson = json.load(f)
add_isochrones(m, biking_geojson, "Biking", "#137815", "#40cc31")

# Contrôle des couches
folium.LayerControl(collapsed=False).add_to(m)

# Sauvegarde
m.save("isochrone_transport_paris.html")

In [16]:
import os
import json
from shapely.geometry import shape, mapping
from shapely.ops import unary_union

INPUT_FOLDER = (
    "/Users/mazziap/Developer/isochrone_map/isochrone_map/data/geojson_mp_data"
)
OUTPUT_FOLDER = (
    "/Users/mazziap/Developer/isochrone_map/isochrone_map/data/geojson_mp_data_outer"
)

os.makedirs(OUTPUT_FOLDER, exist_ok=True)

for filename in os.listdir(INPUT_FOLDER):
    if not filename.endswith(".geojson"):
        continue

    filepath = os.path.join(INPUT_FOLDER, filename)
    with open(filepath, "r", encoding="utf-8") as f:
        geojson = json.load(f)

    # Convertit chaque feature en Shapely Polygon ou MultiPolygon
    polygons = []
    for feature in geojson["features"]:
        geom = shape(feature["geometry"])
        if isinstance(geom, Polygon):
            polygons.append(geom)
        elif isinstance(geom, MultiPolygon):
            polygons.extend(list(geom.geoms))

    # Fusionne toutes les bandes pour n'avoir qu'une seule forme extérieure
    merged = unary_union(polygons)

    # S'assure que le résultat est un MultiPolygon
    if isinstance(merged, Polygon):
        merged = MultiPolygon([merged])

    # Crée un nouveau GeoJSON avec juste la forme extérieure
    output_geojson = {
        "type": "FeatureCollection",
        "features": [
            {"type": "Feature", "properties": {}, "geometry": mapping(merged)}
        ],
    }

    output_path = os.path.join(OUTPUT_FOLDER, filename)
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(output_geojson, f, ensure_ascii=False, indent=2)

    print(f"Traitement terminé pour {filename}")

KeyError: 'features'